# Four-Circle example (E4C)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = logging.getLogger("notebook")
logger.setLevel(logging.INFO)

import gi
gi.require_version('Hkl', '5.0')

from hkl.diffract import E4CV
from hkl.util import Lattice

from bluesky import plans as bp
from bluesky import plan_stubs as bps
from bluesky import RunEngine
RE = RunEngine({})
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
RE.subscribe(bec)

from ophyd import Component
from ophyd import PseudoSingle
from ophyd import SoftPositioner

import diffractometer

import pyRestTable

In [2]:
# describe the diffractometer
class FourCircleDiffractometer(diffractometer.DiffractometerMixin, E4CV):
    h = Component(PseudoSingle, '', labels=("hkl", "fourc"))
    k = Component(PseudoSingle, '', labels=("hkl", "fourc"))
    l = Component(PseudoSingle, '', labels=("hkl", "fourc"))

    omega = Component(SoftPositioner, labels=("motor", "fourc"))
    chi   = Component(SoftPositioner, labels=("motor", "fourc"))
    phi   = Component(SoftPositioner, labels=("motor", "fourc"))
    tth   = Component(SoftPositioner, labels=("motor", "fourc"))

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # since this diffractometer uses simulated motors,
        # prime the SoftPositioners (motors) with initial values
        # otherwise, position == None --> describe, etc gets borked
        for axis in (self.omega, self.phi, self.chi, self.tth):
            axis.move(0)

In [3]:
# initialize the calculation engine
fourc = FourCircleDiffractometer('', name='fourc', labels=("diffractometer", "fourc"))
fourc.calc.engine.mode = fourc.engine.modes[0]  # 'bissector' - constrain tth = 2 * omega
fourc.wavelength = 12.3984244 / 8.04  # Cu Kalpha (angstrom, since that is used in hkl)

print(f"{fourc.name} modes: {fourc.engine.modes}")
print(f"selected mode: {fourc.calc.engine.mode}")

fourc modes: ['bissector', 'constant_omega', 'constant_chi', 'constant_phi', 'double_diffraction', 'psi_constant']
selected mode: bissector


In [4]:
# add a sample
fourc.calc.new_sample('Mn3O4/MgO thin film', 
    lattice=Lattice(
        a=5.72, b=5.72, c=9.5, 
        alpha=90.0, beta=90.0, gamma=90.0))


HklSample(name='Mn3O4/MgO thin film', lattice=LatticeTuple(a=5.72, b=5.72, c=9.5, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 1.09845897e+00, -6.72612131e-17, -4.04983304e-17],
       [ 0.00000000e+00,  1.09845897e+00, -4.04983304e-17],
       [ 0.00000000e+00,  0.00000000e+00,  6.61387927e-01]]), reflections=[])

In [5]:
# use two known reflections to define the orientation
r1 = fourc.calc.sample.add_reflection(
    -1.998, -1.994, 4.011,
    position=fourc.calc.Position(
        tth=80.8769, omega=40.6148, chi=0.647, phi=-121.717))
r2 = fourc.calc.sample.add_reflection(
    -0.997, -0.997, 2.009,
    position=fourc.calc.Position(
        tth=28.695, omega=14.4651, chi=-48.8860, phi=-88.758))
fourc.calc.sample.compute_UB(r1, r2)

1

In [6]:
# apply some constraints
print("existing constraints")
fourc.showConstraints()

diffractometer_constraints = {
    # axis: diffractometer.AxisConstraints(lo_limit, hi_limit, value, fit)
    "omega": diffractometer.AxisConstraints(-120, 150, 0, True),
    "chi": diffractometer.AxisConstraints(-150, 120, 0, True),
    # "phi": diffractometer.AxisConstraints(0, 0, 0, False),
    "tth": diffractometer.AxisConstraints(-10, 142, 0, True),
}

fourc.applyConstraints(diffractometer_constraints)
fourc.showConstraints()


existing constraints
===== ========= ========== ===== ====
axis  low_limit high_limit value fit 
===== ========= ========== ===== ====
omega -180.0    180.0      0.0   True
chi   -180.0    180.0      0.0   True
phi   -180.0    180.0      0.0   True
tth   -180.0    180.0      0.0   True
===== ========= ========== ===== ====

===== =================== ================== ===== ====
axis  low_limit           high_limit         value fit 
===== =================== ================== ===== ====
omega -119.99999999999999 150.0              0.0   True
chi   -150.0              119.99999999999999 0.0   True
phi   -180.0              180.0              0.0   True
tth   -10.0               142.0              0.0   True
===== =================== ================== ===== ====



In [7]:
print(fourc.forwardSolutionsTable(
    (
        (0, 0, 0.5),
        (0, 0, 1),
        (0, 0, 1.5)
    ),
    full=True
))

=========== ======== ======== ========= ========= ========
(hkl)       solution omega    chi       phi       tth     
=========== ======== ======== ========= ========= ========
(0, 0, 0.5) 0        2.32262  -25.72670 -78.18577 4.64525 
(0, 0, 0.5) 1        -2.32262 25.72670  101.81423 -4.64525
(0, 0, 1)   0        4.64907  -25.72670 -78.18577 9.29815 
(0, 0, 1)   1        -4.64907 25.72670  101.81423 -9.29815
(0, 0, 1.5) 0        6.98324  -25.72670 -78.18577 13.96647
=========== ======== ======== ========= ========= ========



In [8]:
detectors = [
    fourc.h, fourc.k, fourc.l,
    # fourc.omega, fourc.chi, fourc.phi, fourc.tth,
    # I0Mon, diode, scint,
    ]

In [9]:
bec.disable_plots()
RE(bp.scan(detectors, fourc.l, 0.5, 1.5, 11))

Transient Scan ID: 1     Time: 2020-02-26 10:16:08
Persistent Unique Scan ID: 'f6663b5a-bc0e-4b10-b95d-42147e54c884'
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |    fourc_l |    fourc_h |    fourc_k |
+-----------+------------+------------+------------+------------+
|         1 | 10:16:08.7 |      0.500 |     -0.000 |     -0.000 |
|         2 | 10:16:08.7 |      0.600 |     -0.000 |      0.000 |
|         3 | 10:16:08.7 |      0.700 |     -0.000 |     -0.000 |
|         4 | 10:16:08.8 |      0.800 |     -0.000 |      0.000 |
|         5 | 10:16:08.8 |      0.900 |     -0.000 |      0.000 |
|         6 | 10:16:08.8 |      1.000 |     -0.000 |      0.000 |
|         7 | 10:16:08.8 |      1.100 |     -0.000 |      0.000 |
|         8 | 10:16:08.8 |      1.200 |     -0.000 |     -0.000 |
|         9 | 10:16:08.9 |      1.300 |     -0.000 |      0.000 |
|        10 | 10:16:08.9 |      1.400 |     -0.000 |     -0.000 |
|  

('f6663b5a-bc0e-4b10-b95d-42147e54c884',)

In [10]:
fourc.applyConstraints({"omega": diffractometer.AxisConstraints(-0, 150, 0, True)})
print(fourc.forwardSolutionsTable(
    (
        (0, 0, 0.5),
        (0, 0, 1),
        (0, 0, 1.5)
    ),
    full=True
))


=========== ======== ======= ========= ========= ========
(hkl)       solution omega   chi       phi       tth     
=========== ======== ======= ========= ========= ========
(0, 0, 0.5) 0        2.32262 -25.72670 -78.18577 4.64525 
(0, 0, 1)   0        4.64907 -25.72670 -78.18577 9.29815 
(0, 0, 1.5) 0        6.98324 -25.72670 -78.18577 13.96647
=========== ======== ======= ========= ========= ========



In [12]:
fourc.undoLastConstraints()
print("solutions with prior constraints")
print(fourc.forwardSolutionsTable(
    (
        (0, 0, 0.5),
        (0, 0, 1),
        (0, 0, 1.5)
    ),
    full=True
))
fourc.resetConstraints()
print("solutions with original constraints")
print(fourc.forwardSolutionsTable(
    (
        (0, 0, 0.5),
        (0, 0, 1),
        (0, 0, 1.5)
    ),
    full=True
))

solutions with prior constraints
=========== ======== ========== ========== ========= =========
(hkl)       solution omega      chi        phi       tth      
=========== ======== ========== ========== ========= =========
(0, 0, 0.5) 0        2.32262    -25.72670  -78.18577 4.64525  
(0, 0, 0.5) 1        -2.32262   25.72670   101.81423 -4.64525 
(0, 0, 0.5) 2        -2.32262   154.27330  -78.18577 -4.64525 
(0, 0, 0.5) 3        2.32262    -154.27330 101.81423 4.64525  
(0, 0, 0.5) 4        -177.67738 25.72670   101.81423 4.64525  
(0, 0, 0.5) 5        -177.67738 154.27330  -78.18577 4.64525  
(0, 0, 1)   0        4.64907    -25.72670  -78.18577 9.29815  
(0, 0, 1)   1        -4.64907   25.72670   101.81423 -9.29815 
(0, 0, 1)   2        -4.64907   154.27330  -78.18577 -9.29815 
(0, 0, 1)   3        4.64907    -154.27330 101.81423 9.29815  
(0, 0, 1)   4        -175.35093 25.72670   101.81423 9.29815  
(0, 0, 1)   5        -175.35093 154.27330  -78.18577 9.29815  
(0, 0, 1.5) 0        6